# Pydantic is all you need: Jason Liu

https://www.youtube.com/watch?v=yj-wSRJwrrc&list=WL&index=5

## 1- Introduction to Structured Prompting

#### Step1: Import and Patch the Module

In [19]:
# Import libraries and apply the patch function to the OpenAI module. This exposes new functionality with the response_model parameter..
import os
import instructor
import openai
from openai import OpenAI
from pydantic import BaseModel

# Load environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

client = instructor.patch(OpenAI())

Our instructor.patch for the OpenAI class introduces three key enhancements:

- Response Mode: Specify a Pydantic model to streamline data extraction.
- Max Retries: Set your desired number of retry attempts for requests.
- Validation Context: Provide a context object for enhanced validator access. A Glimpse into Instructor's Capabilities

#### Step 2: Define the Pydantic Model

In [20]:
# Create a Pydantic model to define the structure of the data you want to extract.
# This model will map directly to the information in the prompt.

class ProductDetail(BaseModel):
    collection: str
    category: str
    sales_last_month: float
    inventory_count: int

#### Step 3: Extract Data with ChatCompletion

- Use the client.chat.completions.create method to send a prompt and extract the data into the Pydantic object.
- The response_model parameter specifies the Pydantic model to use for extraction.
- Its helpful to annotate the variable with the type of the response model. which will help your IDE provide autocomplete and spell check.

In [21]:
user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=ProductDetail,
    messages=[{"role": "user", "content": "Oviedo from coffee tables category are sold 15000 euros last month, the inventory was 40 units at the beginning of the month."}]
)

In [22]:
assert user.collection == "Oviedo"
assert user.category == "coffee tables"
assert user.sales_last_month == 15000
assert user.inventory_count == 40

#### Step 4: Validate the Extracted Data

In [23]:
from pydantic import BaseModel, ValidationError, BeforeValidator
from typing_extensions import Annotated
from instructor import llm_validator

class QuestionAnswer(BaseModel):
    question: str
    answer: Annotated[
        str,
        BeforeValidator(llm_validator("don't say objectionable things"))
    ]

try:
    qa = QuestionAnswer(
        question="What is the meaning of life?",
        answer="The meaning of life is to be evil and steal",
    )
except ValidationError as e:
    print(e)

1 validation error for QuestionAnswer
answer
  Assertion failed, The statement promotes objectionable behavior. [type=assertion_error, input_value='The meaning of life is to be evil and steal', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/assertion_error


- Nothing special, just control flow

- Handling missing data: Program with NLP instead of chaining

- Reuse components: work time, leasure time. If not parsed correctly, add chain of thoughts

- Add "chain of thought" to specific components

- Extract arbitrary values

- Extract Entities and relationships

- **Structured Prompting lets you define components**: Not only model represent the prompt, and the data, we can attach behaviors with methods

- **Structured Prompting outputs data structures**: Represent knowledge, workflows, and plans that can be processed by downstream computer system